In [1]:
import NeuroSAT
import torch
import pickle
import explainer_NeuroSAT
import datasetLoader

In [2]:
opts = {
        'out_dir': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/data/dataset_train_8_size4000',
        'logging': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/log/dataset_train_8_size4000.log',
        'reals_dir': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/data/dataset_train_8_size4000_reals',
        'gt_edges_per_problem': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/data/dataset_train_8_size4000_gt_edges_per_problem',
        'n_pairs': 100,  # Anzahl der zu generierenden Paare
        'min_n': 8,
        'max_n': 8,
        'p_k_2': 0.3,
        'p_geo': 0.4,
        'max_nodes_per_batch': 4000,
        'one_pair': False,
        'emb_dim': 128,
        'iterations': 26,
    }

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Smaller problem

In [11]:
opts = {
        'out_dir': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/data/dataset_train_8_size500',
        'logging': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/log/dataset_train_8_size500.log',
        'reals_dir': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/data/dataset_train_8_size500_reals',
        'gt_edges_per_problem': '/Users/trist/Documents/Bachelor-Thesis/NeuroSAT/test/files/data/dataset_train_8_size500_gt_edges_per_problem',
        'n_pairs': 20,  # Anzahl der zu generierenden Paare
        'min_n': 8,
        'max_n': 8,
        'p_k_2': 0.3,
        'p_geo': 0.4,
        'max_nodes_per_batch': 500,
        'one_pair': False,
        'emb_dim': 128,
        'iterations': 26,
    }

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Generate GT for problem

In [3]:
data = datasetLoader.loadSatBatches(opts['out_dir'])

datasetLoader.storeGTforData(data[-1], opts['min_n'], opts['out_dir'])

#with open(opts['reals_dir'], 'rb') as file:
#        reals = pickle.load(file)

In [3]:
from torch_geometric import seed
import utils
import wandb
import torch.nn.functional as fn
from torcheval.metrics.functional import binary_auroc
from collections import defaultdict


def trainExplainer (datasetName, opts, save_model=False, wandb_project="Explainer-NeuroSAT", runSeed=None) :
    if runSeed is not None: seed.seed_everything(runSeed)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Check valid dataset name
    configOG = utils.loadConfig(datasetName)
    if configOG == -1:
        return
    
    params = configOG['params']
    graph_task = params['graph_task']
    epochs = params['epochs']
    t0 = params['t0']
    tT = params['tT']
    sampled_graphs = params['sampled_graphs']
    coefficient_size_reg = params['coefficient_size_reg']
    coefficient_entropy_reg = params['coefficient_entropy_reg']
    coefficient_L2_reg = params['coefficient_L2_reg']
    coefficient_consistency = params['coefficient_consistency']
    num_explanation_edges = params['num_explanation_edges']
    lr_mlp = params['lr_mlp']

    wandb.init(project=wandb_project, config=params)

    hidden_dim = 64 # Make loading possible
    clip_grad_norm = 2 # Make loading possible
    min_clip_value = -2
    
    with open(opts['out_dir'], 'rb') as file:
        data = pickle.load(file)
    
    # TODO: Split data into train and test
    # TODO: !!! each data consist of multiple problems !!! -> Extract singular problems for calculating the loss!
    dataset = data
    
    eval_problem = data[-1]
    
    with open(opts['reals_dir'], 'rb') as file:
        reals = pickle.load(file)
        
    with open(opts['gt_edges_per_problem'], 'rb') as file:
        gt_edges_per_problem = pickle.load(file)
    

    downstreamTask = NeuroSAT.NeuroSAT(opts=opts,device=device)
    checkpoint = torch.load(f"models/neurosat_sr10to40_ep1024_nr26_d128_last.pth.tar", weights_only=True, map_location=device)
    downstreamTask.load_state_dict(checkpoint['state_dict'])

    mlp = explainer_NeuroSAT.MLP_SAT(GraphTask=graph_task).to(device)
    wandb.watch(mlp, log= "all", log_freq=2, log_graph=False)

    mlp_optimizer = torch.optim.Adam(params = mlp.parameters(), lr = lr_mlp)

    downstreamTask.eval()
    for param in downstreamTask.parameters():
        param.requires_grad = False


    training_iterator = dataset
    
    for epoch in range(0, epochs) :
        mlp.train()
        mlp_optimizer.zero_grad()

        temperature = t0*((tT/t0) ** ((epoch+1)/epochs))
        
        #sampledEdges = 0.0
        #sumSampledEdges = 0.0
        
        #samplePredSum = 0

        for index, content in enumerate(training_iterator):
            # stop training before last batch, used for evaluation
            if index == len(training_iterator)-2: break
            # stop training after first batch
            #if index > 0: break
            node_to_predict = None
            if graph_task: 
                # !! current_problem is really a batch of problems !!
                current_problem = content

            # MLP forward
            # TODO: Implement embeddingCalculation for SAT
            w_ij, unique_clauses, inverse_indices = mlp.forward(downstreamTask, current_problem, nodeToPred=node_to_predict)

            sampleLoss = torch.FloatTensor([0]).to(device)
            loss = torch.FloatTensor([0]).to(device)
            
            pOriginal, _, _, _ = downstreamTask.forward(current_problem)
            pOriginal = fn.softmax(pOriginal, dim=0)
            
            #pOriginal = torch.tensor([1 - pOriginal, pOriginal])
            
            for k in range(0, sampled_graphs):
                edge_ij = mlp.sampleGraph(w_ij, unique_clauses, inverse_indices, temperature)
            
                # TODO: softmax needed for loss, beacuse negative values do not work with log! Need for normalization?
                pSample, _, _, _ = downstreamTask.forward(current_problem, edge_weights=edge_ij)
                pSample = fn.softmax(pSample, dim=0)
                #pSample = torch.tensor([1 - pSample, pSample])
                
                if graph_task:
                    """for sub_problem_idx in range(len(current_problem.is_sat)):
                        # batch_mask needed to differentiate sub_problems in batch of problems for loss
                        # batch_edges cannot simply be divided since sub_problems have different number of edges/clauses
                        # IMPORTANT: batch_size and n variables dependant on data!!
                        batch_mask = utils.get_batch_mask(torch.tensor(current_problem.batch_edges), sub_problem_idx, opts['min_n'], current_problem.n_variables)
                        #(pOriginal[sub_problem_idx] >= 0.5).int()
                        currLoss = mlp.loss(pOriginal[sub_problem_idx], pSample[sub_problem_idx], edge_ij[batch_mask], current_problem.batch_edges[batch_mask], coefficient_size_reg, coefficient_entropy_reg, coefficientConsistency=coefficient_consistency)
                        sampleLoss.add_(currLoss)"""
                    currLoss = mlp.loss(pOriginal, pSample, edge_ij, current_problem.batch_edges, coefficient_size_reg, coefficient_entropy_reg, coefficientConsistency=coefficient_consistency, bce=True)
                    sampleLoss.add_(currLoss)
                    

            loss += sampleLoss / sampled_graphs
            
            #sumSampledEdges += sampledEdges / sampled_graphs

        #print(samplePredSum)
        
        loss = loss / len(training_iterator)
        loss.backward()
        
        mlp_optimizer.step()

        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

        torch.nn.utils.clip_grad_norm_(mlp.parameters(), max_norm=clip_grad_norm)
        
        """for param in mlp.parameters():
            if param.grad is not None:
                param.grad.data = torch.max(param.grad.data, min_clip_value * torch.ones_like(param.grad.data))"""

        mlp.eval()
        
        """if graph_task:
            #TODO: Evaluation for SAT! Needs gt
            meanAuc = evaluation.evaluateNeuroSATAUC(mlp, downstreamTask, data)"""
        
        # Calculate weights and prediction for all sub_problems in eval_problem
        w_ij_eval, unique_clauses_eval, inverse_indices_eval = mlp.forward(downstreamTask, eval_problem, nodeToPred=node_to_predict)
        edge_ij_eval = mlp.sampleGraph(w_ij_eval, unique_clauses_eval, inverse_indices_eval, temperature).detach()
        #pSample_eval, _, _, _ = downstreamTask.forward(eval_problem, edge_weights=edge_ij_eval)
        #pOriginal_eval, _, _, _ = downstreamTask.forward(eval_problem)
        
        auroc_list = []
        
        for current_batch_num in range(len(eval_problem.is_sat)):
            # This can be repeated for each sub_problem
            clauses_current_problem = eval_problem.get_clauses_for_problem(current_batch_num)

            #print(f"Length of sub_problem 0: {len(clauses_current_problem)}")
            # Sub problem 1 Contains literals 11-20, ...
            #print(f"Sub_problem 0 clauses: {clauses_current_problem}")
            
            # Calculate mask for current sub_problem in eval_problem
            eval_batch_mask = utils.get_batch_mask(torch.tensor(eval_problem.batch_edges), batch_idx=current_batch_num, batch_size=opts['min_n'], n_variables=eval_problem.n_variables)
            
            # Edge probabilites for current sub problem in eval_problems
            edge_ij_eval_masked = edge_ij_eval[eval_batch_mask]
            
            sub_problem_edges = eval_problem.batch_edges[eval_batch_mask]
            
            #motif_size = len(gt_mask)
            
            curr_auroc = binary_auroc(edge_ij_eval_masked, reals[current_batch_num])
            auroc_list.append(curr_auroc.item())
            
        
        auroc_tensor = torch.tensor(auroc_list)
        mean_auroc = auroc_tensor.mean()
        if epoch == epochs-1:
            highest_auroc_index = torch.argmax(auroc_tensor)
            highest_mask = utils.get_batch_mask(torch.tensor(eval_problem.batch_edges), batch_idx=highest_auroc_index, batch_size=opts['min_n'], n_variables=eval_problem.n_variables)
            highest_auroc = auroc_tensor[highest_auroc_index]
            highest_edge_ij = edge_ij_eval[highest_mask]
            highest_gt = reals[highest_auroc_index]
            sub_problem_edges_highest = eval_problem.batch_edges[highest_mask]
            
            lowest_auroc_index = torch.argmin(auroc_tensor)
            lowest_mask = utils.get_batch_mask(torch.tensor(eval_problem.batch_edges), batch_idx=lowest_auroc_index, batch_size=opts['min_n'], n_variables=eval_problem.n_variables)
            lowest_auroc = auroc_tensor[lowest_auroc_index]
            lowest_edge_ij = edge_ij_eval[lowest_mask]
            lowest_gt = reals[lowest_auroc_index]
            sub_problem_edges_lowest = eval_problem.batch_edges[lowest_mask]
                
        weights_eval_masked = w_ij_eval[eval_batch_mask]
        print(f"Edge weights for last sub_problem in last problem batch: {weights_eval_masked}")
        
        print(f"Edge probabilites for last sub_problem in last problem batch: {edge_ij_eval_masked}")
        
        #print(f"Prediction for sub_problem: {pOriginal_eval[0]}")
        #print(f"Prediction for sampled sub_problem: {pSample_eval[0]}")
        
        print(f"mean auroc score: {mean_auroc}")
    

        # TODO: VISUALIZE TOPK!
        # print sub_problem 0 with calculated edge weights
        if (epoch+1) % 5 == 0:
            pos = utils.visualize_edge_index_interactive(sub_problem_edges, edge_ij_eval_masked, f"results/replication/seed{runSeed}_vis_edge_ij_{epoch+1}", topK=len(gt_edges_per_problem[-1]))
        
        
        #sumSampledEdges = sumSampledEdges / len(training_iterator)
        #, "val/mean_AUC": meanAuc
        wandb.log({"train/Loss": loss, "val/temperature": temperature, "val/auroc": mean_auroc})

        """for name, param in mlp.named_parameters():
            if param.requires_grad:
                print(f"{name}: {param.grad}")"""
        
    # print sub_problem 0 with calculated edge weights
    pos = utils.visualize_edge_index_interactive(sub_problem_edges, edge_ij_eval_masked, f"results/replication/seed{runSeed}_vis_edge_ij_{epoch+1}", topK=len(gt_edges_per_problem[-1]))
    # print sub_problem 0 with gt
    pos = utils.visualize_edge_index_interactive(sub_problem_edges, reals[-1], f"results/replication/seed{runSeed}_vis_gt", pos)
    
    # TODO: Show difference between topK edges and gt edges -> logical and on topK and gt, visualize
    
    sorted_weights, topk_indices_highest = torch.topk(highest_edge_ij, len(gt_edges_per_problem[highest_auroc_index]))
    mask_topK_highest = torch.zeros_like(highest_edge_ij, dtype=torch.float32)
    mask_topK_highest[topk_indices_highest] = 1
    
    common_edges_highest = torch.logical_and(highest_gt, mask_topK_highest.flatten())
    print(f"Highest individual auroc: {highest_auroc}")
    pos = utils.visualize_edge_index_interactive(sub_problem_edges_highest, highest_edge_ij, f"results/replication/seed{runSeed}_highestAUC", topK=len(gt_edges_per_problem[highest_auroc_index]))
    pos = utils.visualize_edge_index_interactive(sub_problem_edges_highest, highest_gt, f"results/replication/seed{runSeed}_highestAUC_gt")
    pos = utils.visualize_edge_index_interactive(sub_problem_edges_highest, common_edges_highest, f"results/replication/seed{runSeed}_highestAUC_commonEdges")
    
    # TO EVALUATE SATISFIABILITY OF EXPLANATION:
    print(f"Highest AUC example masked edges: {sub_problem_edges_highest[mask_topK_highest.bool()]}")
    
    # Group literals by clause number
    clause_dict = defaultdict(list)

    for literal, clause in sub_problem_edges_highest[mask_topK_highest.bool()]:
        literal = int(literal)
        clause = int(clause)
        if literal >= eval_problem.n_variables:
            literal = -(literal - eval_problem.n_variables)
        clause_dict[clause].append(literal)

    # Convert to list of clauses (optional: sort by clause number)
    clauses = list(clause_dict.values())
    
    print(clauses)
    
    
    sorted_weights, topk_indices_lowest = torch.topk(lowest_edge_ij, len(gt_edges_per_problem[lowest_auroc_index]))
    mask_topK_lowest = torch.zeros_like(lowest_edge_ij, dtype=torch.float32)
    mask_topK_lowest[topk_indices_lowest] = 1
    
    common_edges_lowest = torch.logical_and(lowest_gt, mask_topK_lowest.flatten())
    print(f"Lowest individual auroc: {lowest_auroc}")
    pos = utils.visualize_edge_index_interactive(sub_problem_edges_lowest, lowest_edge_ij, f"results/replication/seed{runSeed}_lowestAUC", topK=len(gt_edges_per_problem[lowest_auroc_index]))
    pos = utils.visualize_edge_index_interactive(sub_problem_edges_lowest, lowest_gt, f"results/replication/seed{runSeed}_lowestAUC_gt")
    pos = utils.visualize_edge_index_interactive(sub_problem_edges_lowest, common_edges_lowest, f"results/replication/seed{runSeed}_lowestAUC_commonEdges")
    
        
    #if save_model == "True":
    #    torch.save(mlp.state_dict(), f"models/explainer_{dataset}_{meanAuc}_{wandb.run.name}")

    wandb.finish()
    
    return mlp, downstreamTask

In [4]:
datasetName="NeuroSAT"

In [5]:
for i in range(1):
    mlp, downstreamTask = trainExplainer(datasetName,opts, wandb_project="NeuroSAT-HARD-CONSTRAINT-HARD-SAMPLING", runSeed=i)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: tristan-schulz2001 (tristan-schulz2001-tu-dortmund) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1, Loss: 1.7834055423736572
Edge weights for last sub_problem in last problem batch: tensor([-0.0227, -0.0227, -0.0227, -0.0691, -0.0691, -0.0691, -0.0691, -0.0691,
        -0.0246, -0.0246, -0.0246, -0.0246, -0.0246, -0.0525, -0.0525, -0.0525,
        -0.0525, -0.0347, -0.0347, -0.0347, -0.0442, -0.0442, -0.0442, -0.0442,
        -0.0442, -0.0442, -0.0442, -0.0442, -0.0274, -0.0274, -0.0274, -0.0274,
        -0.0512, -0.0512, -0.0512, -0.0512, -0.0472, -0.0472, -0.0472, -0.0264,
        -0.0264, -0.0264, -0.0465, -0.0465, -0.0465, -0.0547, -0.0547, -0.0547,
        -0.0547, -0.0547, -0.0547, -0.0547, -0.0331, -0.0331, -0.0331, -0.0331,
         0.0011,  0.0011, -0.0474, -0.0474, -0.0474, -0.0488, -0.0488, -0.0488,
        -0.0488, -0.0488, -0.0587, -0.0587, -0.0587, -0.0587, -0.0587, -0.0587,
        -0.0545, -0.0545, -0.0545, -0.0545, -0.0453, -0.0453, -0.0453, -0.0453,
        -0.0453, -0.0453, -0.0453, -0.0441, -0.0441, -0.0441, -0.0441, -0.0441,
        -0.0441, -0.0264, -0.

train/Loss,█████▇▇▇▇▇▇▇▆▆▆▅▅▅▅▅▅▅▅▄▃▃▂▁▁▁
val/auroc,▁▃▆▆▇▇▇▇██████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
val/temperature,██▇▇▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁
train/Loss,0.92617
val/auroc,0.52311
val/temperature,1


## Sweeping config

In [ ]:
sweep_config = {
    "method": 'grid',                    # random, grid or Bayesian search
    "metric": {"goal": "maximize", "name": "val/auroc"},
    "parameters": {
        'bce_loss': {
            'values': [1, 0]
        },
        'epochs': {
            'values': [20]
            },
        'tT': {
            'values': [1.0, 2.0]
            },
        'size_reg': {
            'values': [0.01]
            },
        'entropy_reg': {
            'values': [0.5]
            },
        'consistency_reg': {
            'values': [0.0]
            },
        'lr_mlp':{
            'values': [0.0001, 0.00003]
            },
        'seed':{
            'values': [74, 75]
            },
    },
}   

In [3]:
sweep_id = wandb.sweep(sweep_config, project="Sweep-NeuroSAT-HARD-CONSTRAINT-AUROC")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: 1kd67zlb
Sweep URL: https://wandb.ai/tristan-schulz2001-tu-dortmund/Sweep-NeuroSAT-HARD-CONSTRAINT-AUROC/sweeps/1kd67zlb


In [4]:
import sweepExplainerNeuroSAT

wandb.agent(sweep_id, sweepExplainerNeuroSAT.trainExplainer)

wandb: Agent Starting Run: bvdp5q2l with config:
wandb: 	bce_loss: 1
wandb: 	consistency_reg: 0
wandb: 	entropy_reg: 0
wandb: 	epochs: 20
wandb: 	lr_mlp: 0.0001
wandb: 	seed: 74
wandb: 	size_reg: 0.01
wandb: 	tT: 1
wandb: Currently logged in as: tristan-schulz2001 (tristan-schulz2001-tu-dortmund) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch 1, Loss: 9.160676956176758
mean auroc score: 0.4891628623008728
Epoch 2, Loss: 9.113851547241211
mean auroc score: 0.490462064743042
Epoch 3, Loss: 9.038740158081055
mean auroc score: 0.5019503235816956
Epoch 4, Loss: 8.95941162109375
mean auroc score: 0.5136950016021729
Epoch 5, Loss: 8.895312309265137
mean auroc score: 0.5189204812049866
Epoch 6, Loss: 8.812456130981445
mean auroc score: 0.5238277912139893
Epoch 7, Loss: 8.747904777526855
mean auroc score: 0.5249457359313965
Epoch 8, Loss: 8.69434642791748
mean auroc score: 0.527639627456665
Epoch 9, Loss: 8.59929370880127
mean auroc score: 0.5281552672386169
Epoch 10, Loss: 8.554255485534668
mean auroc score: 0.5251675248146057
Epoch 11, Loss: 8.43028736114502
mean auroc score: 0.5260509252548218
Epoch 12, Loss: 8.369158744812012
mean auroc score: 0.532546877861023
Epoch 13, Loss: 8.229881286621094
mean auroc score: 0.5343207716941833
Epoch 14, Loss: 8.135065078735352
mean auroc score: 0.5352094769477844
Epoch 15, Loss: 8.0037

train/Loss,███▇▇▇▆▆▆▆▅▅▅▄▄▃▃▂▁▁
val/auroc,▁▁▃▄▅▆▆▆▆▆▆▇▇▇███▇▇▇
val/highest_individual_auroc,▁
val/temperature,█▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁
train/Loss,7.25013
val/auroc,0.53488
val/highest_individual_auroc,0.74816
val/temperature,1


wandb: Agent Starting Run: v2gvd476 with config:
wandb: 	bce_loss: 1
wandb: 	consistency_reg: 0
wandb: 	entropy_reg: 0
wandb: 	epochs: 20
wandb: 	lr_mlp: 0.0001
wandb: 	seed: 74
wandb: 	size_reg: 0.01
wandb: 	tT: 2


Epoch 1, Loss: 9.153109550476074
mean auroc score: 0.4891628623008728
Epoch 2, Loss: 9.101724624633789
mean auroc score: 0.490462064743042
Epoch 3, Loss: 9.028517723083496
mean auroc score: 0.5018190741539001
Epoch 4, Loss: 8.955745697021484
mean auroc score: 0.5133092403411865
Epoch 5, Loss: 8.899576187133789
mean auroc score: 0.5193172693252563
Epoch 6, Loss: 8.831525802612305
mean auroc score: 0.5240538120269775
Epoch 7, Loss: 8.782008171081543
mean auroc score: 0.5254025459289551
Epoch 8, Loss: 8.74250602722168
mean auroc score: 0.5278806686401367
Epoch 9, Loss: 8.678144454956055
mean auroc score: 0.5286290645599365
Epoch 10, Loss: 8.64754867553711
mean auroc score: 0.5262126922607422
Epoch 11, Loss: 8.565909385681152
mean auroc score: 0.5249691009521484
Epoch 12, Loss: 8.525544166564941
mean auroc score: 0.5329347252845764
Epoch 13, Loss: 8.443901062011719
mean auroc score: 0.5357378125190735
Epoch 14, Loss: 8.382349967956543
mean auroc score: 0.5350569486618042
Epoch 15, Loss: 8.

...

train/Loss,████▇▇▇▇▇▆▆▅▅▅▄▃▃▂▂▁
val/auroc,█▆▅▆▄▂▁▁▁▂▂▁▃▂▁▁▂▂▂▃
val/highest_individual_auroc,▁
val/temperature,█▇▇▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁
train/Loss,8.7816
val/auroc,0.49118
val/highest_individual_auroc,0.63425
val/temperature,1


wandb: Agent Starting Run: g0iel8cb with config:
wandb: 	bce_loss: 0
wandb: 	consistency_reg: 0
wandb: 	entropy_reg: 5
wandb: 	epochs: 20
wandb: 	lr_mlp: 3e-05
wandb: 	seed: 75
wandb: 	size_reg: 0.01
wandb: 	tT: 2


Epoch 1, Loss: 9.786530494689941
mean auroc score: 0.5006551146507263
Epoch 2, Loss: 9.77003002166748
mean auroc score: 0.497092604637146
Epoch 3, Loss: 9.763692855834961
mean auroc score: 0.49549561738967896
Epoch 4, Loss: 9.747174263000488
mean auroc score: 0.49625787138938904
Epoch 5, Loss: 9.723962783813477
mean auroc score: 0.4932652711868286
Epoch 6, Loss: 9.705619812011719
mean auroc score: 0.4906492829322815
Epoch 7, Loss: 9.680570602416992
mean auroc score: 0.4890878200531006
Epoch 8, Loss: 9.654205322265625
mean auroc score: 0.4884850084781647
Epoch 9, Loss: 9.640008926391602
mean auroc score: 0.48869019746780396
Epoch 10, Loss: 9.609794616699219
mean auroc score: 0.489568829536438
Epoch 11, Loss: 9.564580917358398
mean auroc score: 0.48990902304649353
Epoch 12, Loss: 9.544339179992676
mean auroc score: 0.4887602627277374
Epoch 13, Loss: 9.512039184570312
mean auroc score: 0.49063101410865784
Epoch 14, Loss: 9.495162963867188
mean auroc score: 0.4893478453159332
Epoch 15, Los

train/Loss,████▇▇▇▆▆▆▅▅▅▅▄▃▃▂▂▁
val/auroc,█▆▅▆▄▃▂▁▁▂▂▂▃▂▂▁▂▂▂▃
val/highest_individual_auroc,▁
val/temperature,█▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁▁
train/Loss,9.20105
val/auroc,0.49153
val/highest_individual_auroc,0.63425
val/temperature,2


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
